Restructure CSV to DF
JFranco | 25 SEP 2023
This notebook will take in CSV files FISH-QUANT analysis, retrieved from a directory, and build a master dataframe that can be used to plot the results from multiple animals. This master dataframe will be saved as a new dataframe which can be used to plot results either in this notebook, another notebook, or imported into Excel or Prism for plotting. 

In [178]:
#                           *** LIBRARIES ***
import numpy as np
import os
import matplotlib.image
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [179]:
#  *** WHERE TO GET AND SAVE DATA ***
dirMain = "/Users/joyfranco/Dropbox (Partners HealthCare)/JF_Shared/Data/FromOthers/Cynthia/PlotGeneration/"

# Get a list of the contents in that folder
os.chdir(dirMain)
files = os.listdir()
print(files)

['hcr3animal1_q4mid_647cmaf_SPOTS_SUMMARY.csv', 'hcr3animal4_q4mid_647cmaf_SPOTS_SUMMARY.csv', '.DS_Store', 'hcr3animal5_q2mid_488calb2_SPOTS_SUMMARY.csv', 'hcr3animal1_q4mid_488calb2_SPOTS_SUMMARY.csv', 'hcr3animal4_q4mid_546mafb_SPOTS_SUMMARY.csv', 'hcr3animal1_q4mid_546mafb_SPOTS_SUMMARY.csv', 'hcr3animal3_q4mid_488calb2_SPOTS_SUMMARY.csv', 'hcr3animal5_q2mid_647cmaf_SPOTS_SUMMARY.csv', 'hcr3animal3_q4mid_647cmaf_SPOTS_SUMMARY.csv', 'hcr3animal3_q4mid_546mafb_SPOTS_SUMMARY.csv', 'hcr3animal5_q2mid_546mafb_SPOTS_SUMMARY.csv', 'hcr3animal4_q4mid_488calb2_SPOTS_SUMMARY.csv']


In [180]:
#  *** COMPILE THE INFORMATION FROM THE VARIOUS FILES INTO ONE DF ***
dfAll = pd.DataFrame()  

for file in files:
    # Check that the file is one that we want
    if "SPOTS_SUMMARY.csv" in file:
        # Get key metadata from filename using indices -- not my preferred method because it's dirty, but it's quick
        animalID = file[10:11]
        channel = file[18:21]   
        target = file[21:25]

        # Load the csv file as a dataframe
        dfCSV = pd.read_csv(dirMain+file)

        # Get the column headings
        col = dfCSV.columns[0].split(";")
        
        for index, row in dfCSV.iterrows():
            # Get raw data and compile into dataframe based on semicolon delimiter
            entry = row[0].split(";")
            dfTemp = pd.DataFrame(np.array([entry]),columns=col)
            dfTemp.drop(columns=dfTemp.columns[0], axis=1,  inplace=True)

            # Add additional fields to help with plotting
            dfTemp['animalID'] = animalID
            dfTemp['channel'] = channel
            dfTemp['target'] = target
            dfTemp['roiID'] = str(animalID+"_"+entry[0]) # Found big issue with excel dropping last zero 
            # Append to main df
            dfAll = pd.concat([dfAll, dfTemp])

dfAll.reset_index(drop=True, inplace=True)

In [181]:
#   *** Save compiled DF to file ***
dfAll.to_csv(dirMain+"RestructuredDFFromCSV_All.csv")

#   *** Reload the dataframe to help with subsetting
#  Ran into a weird issue where .loc returned an empty dataframe and didn't have time to troubleshoot
dfAll = pd.read_csv(dirMain+"RestructuredDFFromCSV_All.csv")

In [197]:
#  *** REORGANIZE THE DATAFRAME FOR EASY PLOTTING ***
# Get list of all unique info 
rois = dfAll['roiID'].unique()
targets = dfAll['target'].unique()

# Iterate through each unique ROI and build relevant info for new dataframe 
dfROIsAll = pd.DataFrame()
for roi in rois:
    dfSS = dfAll[dfAll['roiID']==roi]
    channels = dfSS['channel'].unique()
    animalID = dfSS['animalID'].iloc[0]

    # Iterate through the channels for this ROI
    colsAll = []
    chArr = []
    for chan in channels:
        
        # Get the values for each column
        chCA = dfSS[dfSS['channel']==chan].iloc[0]['cell_area']   
        chNA = dfSS[dfSS['channel']==chan].iloc[0]['nuc_area']
        chNB = dfSS[dfSS['channel']==chan].iloc[0]['nb_rna']
        chNIN = dfSS[dfSS['channel']==chan].iloc[0]['nb_rna_in_nuc']
        chNON = dfSS[dfSS['channel']==chan].iloc[0]['nb_rna_out_nuc']
        chTAR = dfSS[dfSS['channel']==chan].iloc[0]['target']
        chInfo = [chTAR, chCA, chNA, chNB, chNIN, chNON]
        chArr = chArr+chInfo
        
        # Setup all of the columns for this roi and 
        cols = [str(chan)+'_'+chTAR, 'cell_area_'+chTAR, 'nuc_area_'+chTAR, 'nb_rna_'+chTAR, 
                'nb_rna_in_nuc_'+chTAR, 'nb_rna_out_nuc_'+chTAR]
        colsAll = colsAll+cols
        
    # Combine all info for the ROI into a single dataframe     
    dfROI = pd.DataFrame(np.array([chArr]), columns = colsAll)
    
    # Add key metadata
    dfROI.insert(loc = 0,
          column = 'roiID',
          value = roi)
    dfROI.insert(loc = 0,
          column = 'animalID',
          value = animalID)

    # Add to master list    
    dfROIsAll = pd.concat([dfROIsAll, dfROI])



In [199]:
# Reset index on master list
dfROIsAll.reset_index(drop=True, inplace=True)   
dfROIsAll = dfROIsAll.fillna("0")
#   *** Save All ROIs ***
dfROIsAll.to_csv(dirMain+"ROIInfoAllChannels.csv")